# Network visualization with pyvis

Last updated: 01292024
By: Lauren Liao  
Purpose: visualize and analyze from word similarity matrix 

In [1]:
import networkx as nx
import pandas as pd
import matplotlib.pyplot as plt
from collections import Counter
import numpy as np
import seaborn as sns
import graph_tool.all as gt
import itertools
from pyvis.network import Network

%matplotlib inline

**Data loading**  
the data starts with the preprocessed text (pickled) file  
taking the upper right triangle plus a threshold of 50th quartile
fully connected and take the top 30 for word embeddings set

In [2]:
top30_lst =  pd.read_pickle("../../results/top30_mean_all.pkl")

## Graph creation and analysis

In [5]:
graph_kt = ['stereotype', 'exclusion', 'prejudice', 'superiority', 'experience'
            # 'overt', 'homophobia', 'bias', 'racial_discrimination', 'religion', 
            # 'oppression', 'implicit', 'racism', 'discrimination', 'segregation', 
            # 'perceived', 'sexism', 'stereotyping', 'belief', 
            # 'race', 'racial', 'sexual_orientation',
            # 'systemic', 'gender', 'interpersonal', 'injustice', 'historical', 
            # 'culture', 'cultural', 'construct', 'identity',
            # 'stress', 'stigma', 'deprivation', 'negative', 'disadvantage', 'biological',
            # 'driver', 'psychosocial', 'wealth', 'consequence', 
            # 'affecting', 'environmental', 'economic', 'vulnerability', 
            # 'food_security', 'distribution', 'underlying', 'poverty',
            # 'societal', 'norm', 'sociopolitical', 'fundamental', 
            # 'structural', 'influence',
            # 'difference', 'sex', 'class', 'ethnicity', 'disparity',
            # 'power'
           ]

In [14]:
graph_kt = ['stereotype', 'exclusion', 'prejudice', 'superiority', 'experience', 
            'overt', 'homophobia', 'bias', 'racial_discrimination', 'religion', 
            'oppression', 'implicit', 'racism', 'discrimination', 'segregation', 
            'perceived', 'sexism', 'stereotyping', 'belief', 
            'race', 'racial', 'sexual_orientation',
            'systemic', 'gender', 'interpersonal', 'injustice', 'historical', 
            'culture', 'cultural', 'construct', 'identity',
            'stress', 'stigma', 'deprivation', 'negative', 'disadvantage', 'biological',
            'driver', 'psychosocial', 'wealth', 'consequence', 
            'affecting', 'environmental', 'economic', 'vulnerability', 
            'food_security', 'distribution', 'underlying', 'poverty',
            'societal', 'norm', 'sociopolitical', 'fundamental', 
            'structural', 'influence',
            'difference', 'sex', 'class', 'ethnicity', 'disparity',
            'power']

In [6]:
node_labels = {word:i for i, word in enumerate(set(graph_kt))}
sub_30lst = [(x, y) for x,y in top30_lst if x in set(node_labels)]


vis_kt = list(itertools.product(graph_kt, graph_kt))
vis_kt = [(v[0],v[1]) for v in vis_kt if v[0] != v[1]]

wt = []

for word1, word2 in vis_kt:
    word_lst = [y for x,y in sub_30lst if x == word1][0]
    if word2 in word_lst:
        wt.append((1-word_lst[word2])*100000)
    else: wt.append(np.nan)
    
vis_kt_wt = [(v[0], v[1], wt[i]) for i,v in enumerate(vis_kt) if np.isnan(wt[i]) == False]

In [61]:
vis_kt_wt

[('stereotype', 'superiority', 2797.9906400044774),
 ('exclusion', 'stereotype', 6214.667757352188),
 ('exclusion', 'prejudice', 8796.621958414718),
 ('exclusion', 'superiority', 7662.199536959335),
 ('prejudice', 'stereotype', 6641.444961229959),
 ('prejudice', 'exclusion', 8796.621958414718),
 ('prejudice', 'superiority', 8562.878966331478),
 ('superiority', 'stereotype', 2797.9906400044774),
 ('experience', 'stereotype', 20285.836656888325),
 ('experience', 'exclusion', 24954.238136609398),
 ('experience', 'prejudice', 14345.261057217916)]

In [53]:
"exclusion" in [y for x,y in sub_30lst if x == "prejudice"][0]
"prejudice" in [y for x,y in sub_30lst if x == "exclusion"][0]
[(x,y,z) for x,y,z in vis_kt_wt if (((x == "prejudice")|(x == "exclusion")) & 
                                    ((y == "prejudice")|(y == "exclusion")))]

True

In [30]:
got_net = Network(height="750px", width="100%", font_color="black", notebook = True)
edge_data = vis_kt_wt
got_net.barnes_hut()
for e in edge_data:
    src = e[0]
    dst = e[1]
    w = e[2]

    got_net.add_node(src, src, title=src)
    got_net.add_node(dst, dst, title=dst)
    got_net.add_edge(src, dst, mass=w)

neighbor_map = got_net.get_adj_list()

# add neighbor data to node hover data
for node in got_net.nodes:
    node["title"] += " Neighbors:<br>" + "<br>".join(neighbor_map[node["id"]])
    node["value"] = len(neighbor_map[node["id"]])
    node["font"] = {"size": 200, "color": "black", "font.family": "palatino"}

got_net.show("vis1.html")

vis1.html
